In [1]:
import pandas as pd
import numpy as np
import csv
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
ALL_SONG_PATH="D:/CSE519/Project/billboard_w_new_features_v7.csv"
FEATURE_ROOT="D:/CSE519/Project/song_feature/"
NEW_FEATURE_CSV_PATH="D:/CSE519/Project/allsong_feature.csv"

# Read all song information and drop duplicates

In [4]:
allSong= pd.read_csv(ALL_SONG_PATH, encoding='latin-1')
allSong.sort_values(by=["weekid"], ascending=True)
allSong.drop_duplicates(subset=['songid'], keep='first', inplace=True )
allSong.shape

(27278, 17)

# preprocess the all song data frame  with "/" in song name 

In [5]:
def split_funct(x):
    song=x['song']
    if "/" in song:
        return song.split("/")
    else:
        return []
allSong['split']= allSong.apply(lambda x: split_funct(x), axis=1)

# double check the split function works

In [6]:
withSplit=allSong['split'].map(lambda x: len(x)>=2)
print(allSong[withSplit].shape)
withNoSplit=allSong['split'].map(lambda x: x==[])
print(allSong[withNoSplit].shape)
allSong.head()
withSlash = allSong['song'].map(lambda x: "/" in x)
print(allSong[withSlash].shape)
withNoSlash=allSong['song'].map(lambda x: "/" not in x)
print(allSong[withNoSlash].shape)

(263, 18)
(27015, 18)
(263, 18)
(27015, 18)


# All features that needs to merge

In [7]:
def process_feature_name(list_feature_name):
    new_list_feature=[]
    for file in list_feature_name:
        a=file.replace('D:/CSE519/Project/song_feature\songs-','')
        b=a.replace(".csv",'')     
        new_list_feature.append(b)
    return new_list_feature

list_feature_name=glob.glob(FEATURE_ROOT+"*.csv")
print(len(list_feature_name))
feature_name=process_feature_name(list_feature_name)
print(len(feature_name))

26
26


In [8]:
def check_split_func(song_row, featureSong, featureArtist, feature_name):
    split_song =song_row['split']
    if ((featureSong in split_song) & (featureArtist==song_row['performer'])):
        song_row[feature_name]=1   
    
def slash_func(featureSong_row,  allSong, feature_name):
    featureSong=featureSong_row['song']
    featureArtist=featureSong_row['artist']
    if "/" not in featureSong:
        if (allSong.loc[ (allSong.song==featureSong)  & (allSong.performer==featureArtist) ].shape[0]!=0):
            allSong.loc[ ((allSong.song == featureSong) & (allSong.performer==featureArtist)) , feature_name] = 1
        else:            
            allSong.apply(lambda x: check_split_func(x, featureSong, featureArtist, feature_name), axis=1)
    if "/" in featureSong:
        featureSongs_array=featureSong.split("/")
        for songs in featureSongs_array:
            if (allSong.loc[(allSong.song==featureSong)  & (allSong.performer==featureArtist)].shape[0]!=0):
                allSong.loc[((allSong.song == featureSong) & (allSong.performer==featureArtist)), feature_name] = 1
            else:            
                allSong.apply(lambda x: check_split_func(x, songs,featureArtist, feature_name), axis=1)    

In [11]:
for idx, feature in enumerate (feature_name):    
    feature_df=pd.read_csv(list_feature_name[idx], encoding='latin-1')
    allSong[feature]=0
    feature_df.apply(lambda x: slash_func(x, allSong,feature), axis=1)
    print(idx)

In [ ]:
allSong.to_csv(NEW_FEATURE_CSV_PATH, index=False)

In [ ]:
allSong.shape

In [9]:
allSong=pd.read_csv(NEW_FEATURE_CSV_PATH,sep='\t')
allSong.drop("Unnamed: 0", inplace=True,axis=1)
allSong.shape

(27278, 44)

In [10]:
allSong.head(1)

,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,...,karaoke,lullabies,mtvawards,phonecall,politicians,sportingevents,tvshow,weddings,wonoscars,wordcup
0,http://www.billboard.com/charts/hot-100/1990-0...,1990-05-05,84,"""B"" Girls",Young And Restless,"""B"" GirlsYoung And Restless",1,NaN,54,15,...,0,0,0,0,0,0,0,0,0,0


# song features and the matches we found

In [11]:
for idx, feature in enumerate (feature_name):    
    feature_df=pd.read_csv(list_feature_name[idx], encoding='latin-1')
    print ("the songs with feature "+ feature+ " "+ str(feature_df.shape))
    allsong_with_feature=allSong.loc[allSong[feature]==1]
    print("the matches that we finally find out "+ str(allsong_with_feature.shape))

the songs with feature artistdied (46, 2)
the matches that we finally find out (22, 44)
the songs with feature christmas (225, 2)
the matches that we finally find out (18, 44)
the songs with feature church (102, 2)
the matches that we finally find out (4, 44)
the songs with feature classical (73, 2)
the matches that we finally find out (0, 44)
the songs with feature commercials (699, 2)
the matches that we finally find out (272, 44)
the songs with feature concerts (35, 2)
the matches that we finally find out (13, 44)
the songs with feature easter (20, 2)
the matches that we finally find out (0, 44)
the songs with feature fourthofjuly (25, 2)
the matches that we finally find out (6, 44)
the songs with feature funerals (39, 2)
the matches that we finally find out (11, 44)
the songs with feature graduations (71, 2)
the matches that we finally find out (38, 44)
the songs with feature grammys (450, 2)
the matches that we finally find out (244, 44)
the songs with feature halloween (35, 2)
th

In [12]:
df_array=[]
for idx, feature in enumerate (feature_name):    
    allsong_with_feature=allSong.loc[allSong[feature]==1]
    df_array.append(allsong_with_feature)

In [13]:
print(feature_name[0])
columns_to_drop=feature_name[1:25]
df_mody=df_array[0].drop(columns_to_drop, axis=1)
df_mody

artistdied


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,artistdied,wordcup
117,http://www.billboard.com/charts/hot-100/1980-1...,1980-11-01,38,(Just Like) Starting Over,John Lennon,(Just Like) Starting OverJohn Lennon,1,NaN,1,22,1980,44,0,0,51,47,0,[],1,0
163,http://www.billboard.com/charts/hot-100/1968-0...,1968-01-27,67,(Sittin' On) The Dock Of The Bay,Otis Redding,(Sittin' On) The Dock Of The BayOtis Redding,1,NaN,1,16,1968,4,0,0,69,62,0,[],1,0
293,http://www.billboard.com/charts/hot-100/1982-1...,1982-10-30,81,1999,Prince,1999Prince,1,NaN,12,30,1982,43,3,6321,66,0,1,[],1,0
442,http://www.billboard.com/charts/hot-100/1965-0...,1965-01-30,70,A Change Is Gonna Come,Sam Cooke,A Change Is Gonna ComeSam Cooke,1,NaN,31,7,1965,4,0,0,64,0,0,[],1,0
3891,http://www.billboard.com/charts/hot-100/2013-0...,2013-06-08,26,Clouds,Zach Sobiech,CloudsZach Sobiech,1,NaN,26,2,2013,23,0,0,36,0,0,[],1,0
5196,http://www.billboard.com/charts/hot-100/1997-1...,1997-12-20,96,Doin' Time,Sublime,Doin' TimeSublime,1,NaN,87,7,1997,51,0,0,53,57,0,[],1,0
5216,http://www.billboard.com/charts/hot-100/1958-1...,1958-11-22,93,Donna,Ritchie Valens,DonnaRitchie Valens,1,NaN,2,23,1958,47,0,0,0,53,0,[],1,0
7009,http://www.billboard.com/charts/hot-100/1995-1...,1995-12-30,10,Free As A Bird,The Beatles,Free As A BirdThe Beatles,1,NaN,6,11,1995,52,0,0,48,0,1,[],1,0
7324,http://www.billboard.com/charts/hot-100/1971-0...,1971-09-11,80,Get It While You Can,Janis Joplin,Get It While You CanJanis Joplin,1,NaN,78,2,1971,36,0,0,42,29,0,[],1,0
9526,http://www.billboard.com/charts/hot-100/1997-0...,1997-04-26,2,Hypnotize,The Notorious B.I.G.,HypnotizeThe Notorious B.I.G.,1,NaN,1,20,1997,17,0,0,77,68,0,[],1,0


In [33]:
columns_to_drop=feature_name[1:25]
allSong.loc[allSong['songid']=="(Sittin' On) The Dock Of The BayOtis Redding"].drop(columns_to_drop, axis=1)

,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,artistdied,wordcup
163,http://www.billboard.com/charts/hot-100/1968-0...,1968-01-27,67,(Sittin' On) The Dock Of The Bay,Otis Redding,(Sittin' On) The Dock Of The BayOtis Redding,1,NaN,1,16,1968,4,0,0,69,62,0,[],1,0


In [37]:
print(feature_name[1])
columns_to_drop=feature_name[2:26]+[feature_name[0]]
print(columns_to_drop)
df_mody=df_array[1].drop(columns_to_drop, axis=1)
df_mody

christmas
['church', 'classical', 'commercials', 'concerts', 'easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,christmas
1024,http://www.billboard.com/charts/hot-100/2000-0...,2000-01-08,83,All I Want For Christmas Is You,Mariah Carey,All I Want For Christmas Is YouMariah Carey,1,NaN,11,19,2000,1,5,4732,85,58,1,[],1
2810,http://www.billboard.com/charts/hot-100/2010-1...,2010-12-18,79,Boots,The Killers,BootsThe Killers,1,NaN,79,1,2010,50,0,0,0,0,0,[],1
3789,http://www.billboard.com/charts/hot-100/2010-1...,2010-12-18,25,Christmas Lights,Coldplay,Christmas LightsColdplay,1,NaN,25,3,2010,50,0,0,69,59,0,[],1
8036,http://www.billboard.com/charts/hot-100/1998-0...,1998-01-10,87,Grandma Got Run Over By A Reindeer,Elmo & Patsy,Grandma Got Run Over By A ReindeerElmo & Patsy,1,NaN,87,1,1998,2,0,0,50,17,0,[],1
9574,http://www.billboard.com/charts/hot-100/1975-1...,1975-12-20,99,I Believe In Father Christmas,Greg Lake,I Believe In Father ChristmasGreg Lake,1,NaN,95,3,1975,51,0,0,0,0,0,[],1
12307,http://www.billboard.com/charts/hot-100/1958-1...,1958-12-20,57,Jingle Bell Rock,Bobby Helms,Jingle Bell RockBobby Helms,1,NaN,29,19,1958,51,5,19369,71,70,0,[],1
12983,http://www.billboard.com/charts/hot-100/2017-0...,2017-01-07,50,Last Christmas,Wham!,Last ChristmasWham!,1,NaN,41,2,2017,1,0,0,75,63,0,[],1
14888,http://www.billboard.com/charts/hot-100/2014-1...,2014-12-06,26,"Mary, Did You Know?",Pentatonix,"Mary, Did You Know?Pentatonix",1,NaN,26,7,2014,49,1,728,0,68,0,[],1
15057,http://www.billboard.com/charts/hot-100/1980-1...,1980-12-27,82,Merry Christmas In The NFL,"Willis ""The Guard"" & Vigorish","Merry Christmas In The NFLWillis ""The Guard"" &...",1,NaN,82,3,1980,52,0,0,0,0,0,[],1
15250,http://www.billboard.com/charts/hot-100/2011-1...,2011-11-05,11,Mistletoe,Justin Bieber,MistletoeJustin Bieber,1,NaN,11,10,2011,44,1,14,77,0,1,[],1


In [38]:
df_BingCrosby=allSong.loc[allSong['performer']=="Bing Crosby"].drop(columns_to_drop, axis=1)
df_BingCrosby

,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,christmas
759,http://www.billboard.com/charts/hot-100/1960-1...,1960-12-10,79,"Adeste Fideles (Oh, Come, All Ye Faithful)",Bing Crosby,"Adeste Fideles (Oh, Come, All Ye Faithful)Bing...",1,NaN,45,3,1960,49,0,0,0,0,0,[],0
19824,http://www.billboard.com/charts/hot-100/1960-1...,1960-12-17,64,Silent Night,Bing Crosby,Silent NightBing Crosby,1,NaN,54,2,1960,50,0,0,45,43,0,[],1
25844,http://www.billboard.com/charts/hot-100/1958-1...,1958-12-20,86,White Christmas,Bing Crosby,White ChristmasBing Crosby,1,NaN,12,13,1958,51,4,364,73,56,0,[],1


In [39]:
print(feature_name[2])
columns_to_drop=feature_name[3:26]+[feature_name[0], feature_name[1]]
print(columns_to_drop)
df_mody=df_array[2].drop(columns_to_drop, axis=1)
df_mody

church
['classical', 'commercials', 'concerts', 'easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,church
12284,http://www.billboard.com/charts/hot-100/1972-1...,1972-12-16,97,Jesus Is Just Alright,The Doobie Brothers,Jesus Is Just AlrightThe Doobie Brothers,1,NaN,35,11,1972,50,0,0,31,48,0,[],1
15413,http://www.billboard.com/charts/hot-100/1972-0...,1972-04-01,78,Morning Has Broken,Cat Stevens,Morning Has BrokenCat Stevens,1,NaN,6,14,1972,13,0,0,63,37,0,[],1
19824,http://www.billboard.com/charts/hot-100/1960-1...,1960-12-17,64,Silent Night,Bing Crosby,Silent NightBing Crosby,1,NaN,54,2,1960,50,0,0,45,43,0,[],1
22711,http://www.billboard.com/charts/hot-100/1974-0...,1974-02-23,71,The Lord's Prayer,Sister Janet Mead,The Lord's PrayerSister Janet Mead,1,NaN,4,13,1974,8,0,0,0,12,0,[],1


In [40]:
print(feature_name[3])
columns_to_drop=feature_name[4:26]+[feature_name[0], feature_name[1], feature_name[2]]
print(columns_to_drop)
df_mody=df_array[3].drop(columns_to_drop, axis=1)
df_mody

classical
['commercials', 'concerts', 'easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,classical


In [55]:
print(feature_name[4])
columns_to_drop=feature_name[5:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3]]
print(columns_to_drop)
df_mody=df_array[4].loc[df_array[4]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody.sort_values(by="year", ascending=True).head(20)

commercials
['concerts', 'easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,commercials
2427,http://www.billboard.com/charts/hot-100/1960-0...,1960-01-16,74,Beyond The Sea,Bobby Darin,Beyond The SeaBobby Darin,1,NaN,6,14,1960,2,0,0,58,63,1,[],1
26310,http://www.billboard.com/charts/hot-100/1960-0...,1960-05-07,97,Wonderful World,Sam Cooke,Wonderful WorldSam Cooke,1,NaN,12,15,1960,18,0,0,65,69,0,[],1
2793,http://www.billboard.com/charts/hot-100/1962-0...,1962-05-26,100,Boom Boom,John Lee Hooker,Boom BoomJohn Lee Hooker,1,NaN,60,10,1962,21,0,0,55,50,0,[],1
26175,http://www.billboard.com/charts/hot-100/1963-0...,1963-06-22,86,Wipe Out,The Surfaris,Wipe OutThe Surfaris,1,NaN,2,30,1963,25,1,1029,43,42,0,[],1
3245,http://www.billboard.com/charts/hot-100/1964-0...,1964-01-25,70,California Sun,The Rivieras,California SunThe Rivieras,1,NaN,5,10,1964,4,0,0,38,0,0,[],1
3547,http://www.billboard.com/charts/hot-100/1965-0...,1965-05-15,85,Catch The Wind,Donovan,Catch The WindDonovan,1,NaN,23,10,1965,19,0,0,58,29,0,[],1
15012,http://www.billboard.com/charts/hot-100/1966-1...,1966-11-12,65,Mellow Yellow,Donovan,Mellow YellowDonovan,1,NaN,2,12,1966,45,0,0,32,51,0,[],1
21336,http://www.billboard.com/charts/hot-100/1966-0...,1966-07-30,90,Sunshine Superman,Donovan,Sunshine SupermanDonovan,1,NaN,1,13,1966,30,0,0,57,0,0,[],1
23393,http://www.billboard.com/charts/hot-100/1966-0...,1966-01-22,74,These Boots Are Made For Walkin',Nancy Sinatra,These Boots Are Made For Walkin'Nancy Sinatra,1,NaN,1,14,1966,3,0,0,65,43,0,[],1
24199,http://www.billboard.com/charts/hot-100/1966-1...,1966-12-03,85,Try A Little Tenderness,Otis Redding,Try A Little TendernessOtis Redding,1,NaN,25,10,1966,48,0,0,62,53,0,[],1


In [50]:
allSong.loc[allSong['performer']=="John Lee Hooker"].drop(columns_to_drop, axis=1)

,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,commercials
2793,http://www.billboard.com/charts/hot-100/1962-0...,1962-05-26,100,Boom Boom,John Lee Hooker,Boom BoomJohn Lee Hooker,1,NaN,60,10,1962,21,0,0,55,50,0,[],1


In [58]:
# feature_name.remove('inmovies')
a=allSong.loc[allSong['inmovies']==1].drop(feature_name, axis=1)
a.loc[a['performer']=="John Lee Hooker"]

,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,inmovies
2793,http://www.billboard.com/charts/hot-100/1962-0...,1962-05-26,100,Boom Boom,John Lee Hooker,Boom BoomJohn Lee Hooker,1,NaN,60,10,1962,21,0,0,55,50,0,[],1


In [15]:
print(feature_name[5])
columns_to_drop=feature_name[6:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4]]
print(columns_to_drop)
df_mody=df_array[5].loc[df_array[5]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

concerts
['easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical', 'commercials']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,concerts
319,http://www.billboard.com/charts/hot-100/1970-0...,1970-07-25,50,25 Or 6 To 4,Chicago,25 Or 6 To 4Chicago,1,NaN,4,20,1970,30,1,5810,61,38,1,[],1
5707,http://www.billboard.com/charts/hot-100/1973-1...,1973-10-20,88,Dream On,Aerosmith,Dream OnAerosmith,1,NaN,6,29,1973,42,1,756,77,62,0,[],1
6029,http://www.billboard.com/charts/hot-100/1991-0...,1991-08-24,47,Enter Sandman,Metallica,Enter SandmanMetallica,1,NaN,16,20,1991,34,0,0,71,39,0,[],1
7010,http://www.billboard.com/charts/hot-100/1974-1...,1974-11-23,87,Free Bird,Lynyrd Skynyrd,Free BirdLynyrd Skynyrd,1,NaN,19,20,1974,47,1,665,69,47,0,[],1
13006,http://www.billboard.com/charts/hot-100/2000-0...,2000-07-29,75,Last Resort,Papa Roach,Last ResortPapa Roach,1,NaN,57,20,2000,30,0,0,79,56,0,[],1
15685,http://www.billboard.com/charts/hot-100/2002-0...,2002-09-07,98,My Friends Over You,New Found Glory,My Friends Over YouNew Found Glory,1,NaN,85,6,2002,36,0,0,61,30,0,[],1
17445,http://www.billboard.com/charts/hot-100/1974-0...,1974-02-23,94,Piano Man,Billy Joel,Piano ManBilly Joel,1,NaN,25,14,1974,8,0,0,69,72,1,[],1
18375,http://www.billboard.com/charts/hot-100/1968-1...,1968-10-12,100,Ride My See-Saw,The Moody Blues,Ride My See-SawThe Moody Blues,1,NaN,61,5,1968,41,0,0,44,23,0,[],1
19136,http://www.billboard.com/charts/hot-100/1972-0...,1972-06-03,88,School's Out,Alice Cooper,School's OutAlice Cooper,1,NaN,7,13,1972,22,0,0,63,0,0,[],1
19319,http://www.billboard.com/charts/hot-100/2003-0...,2003-05-24,76,Seven Nation Army,The White Stripes,Seven Nation ArmyThe White Stripes,1,NaN,76,20,2003,21,0,0,74,55,0,[],1


In [19]:
allSong.loc[allSong['songid']=="25 Or 6 To 4Chicago"]["inmovies"]
#This song is also in movies but our data don't have this information 

319    0
Name: inmovies, dtype: int64

In [26]:
allSong.loc[allSong['song']=='25 Or 6 To 4']

,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,...,karaoke,lullabies,mtvawards,phonecall,politicians,sportingevents,tvshow,weddings,wonoscars,wordcup
319,http://www.billboard.com/charts/hot-100/1970-0...,1970-07-25,50,25 Or 6 To 4,Chicago,25 Or 6 To 4Chicago,1,NaN,4,20,...,0,0,0,0,0,1,0,0,0,0


In [28]:
allSong.loc[allSong['performer']=="The Who"]['all_time_greatest_artist'].tolist()[0]

0

In [32]:
allSong.loc[allSong['songid']=="Piano ManBilly Joel"]['all_time_greatest_artist']

17445    1
Name: all_time_greatest_artist, dtype: int64

In [36]:
print(feature_name[6])
columns_to_drop=feature_name[6:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4], feature_name[5]]
print(columns_to_drop)
df_mody=df_array[6].loc[df_array[6]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

easter
['easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical', 'commercials', 'concerts']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split


In [37]:
print(feature_name[7])
columns_to_drop=feature_name[6:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6]]
print(columns_to_drop)
df_mody=df_array[7].loc[df_array[7]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

fourthofjuly
['easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical', 'commercials', 'concerts', 'easter']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split
1305,http://www.billboard.com/charts/hot-100/2014-0...,2014-07-05,82,American Kids,Kenny Chesney,American KidsKenny Chesney,1,NaN,23,20,2014,27,0,0,67,34,0,[]
1315,http://www.billboard.com/charts/hot-100/2003-1...,2003-12-27,69,American Soldier,Toby Keith,American SoldierToby Keith,1,NaN,28,20,2003,52,0,0,59,42,0,[]
1321,http://www.billboard.com/charts/hot-100/1981-0...,1981-04-25,76,America,Neil Diamond,AmericaNeil Diamond,1,NaN,8,17,1981,17,0,0,49,39,1,[]
4255,http://www.billboard.com/charts/hot-100/2002-0...,2002-06-08,73,"Courtesy Of The Red, White And Blue (The Angry...",Toby Keith,"Courtesy Of The Red, White And Blue (The Angry...",1,NaN,25,20,2002,23,0,0,64,41,0,[]
6644,http://www.billboard.com/charts/hot-100/2010-1...,2010-11-06,57,Firework,Katy Perry,FireworkKaty Perry,1,NaN,1,39,2010,44,0,0,67,72,1,[]


In [38]:
print(feature_name[8])
columns_to_drop=feature_name[7:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6]]
print(columns_to_drop)
df_mody=df_array[8].loc[df_array[8]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

funerals
['fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical', 'commercials', 'concerts', 'easter']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split
1418,http://www.billboard.com/charts/hot-100/1998-1...,1998-12-05,63,Angel,Sarah McLachlan,AngelSarah McLachlan,1,NaN,4,28,1998,49,0,0,66,0,0,[]
1421,http://www.billboard.com/charts/hot-100/1999-1...,1999-11-20,85,Angels,Robbie Williams,AngelsRobbie Williams,1,NaN,53,19,1999,46,0,0,70,0,0,[]
3336,http://www.billboard.com/charts/hot-100/1987-1...,1987-11-07,68,Candle In The Wind,Elton John,Candle In The WindElton John,1,NaN,6,21,1987,45,0,0,55,49,1,[]
10634,http://www.billboard.com/charts/hot-100/1992-1...,1992-11-14,40,I Will Always Love You,Whitney Houston,I Will Always Love YouWhitney Houston,1,NaN,1,29,1992,46,1,6867,74,61,1,[]
10784,http://www.billboard.com/charts/hot-100/2011-0...,2011-04-09,85,If Heaven Wasn't So Far Away,Justin Moore,If Heaven Wasn't So Far AwayJustin Moore,1,NaN,49,20,2011,14,0,0,56,16,0,[]
13173,http://www.billboard.com/charts/hot-100/1970-0...,1970-03-21,6,Let It Be,The Beatles,Let It BeThe Beatles,1,NaN,1,14,1970,12,0,0,77,59,1,[]
15413,http://www.billboard.com/charts/hot-100/1972-0...,1972-04-01,78,Morning Has Broken,Cat Stevens,Morning Has BrokenCat Stevens,1,NaN,6,14,1972,13,0,0,63,37,0,[]
19928,http://www.billboard.com/charts/hot-100/2009-0...,2009-04-25,92,Sissy's Song,Alan Jackson,Sissy's SongAlan Jackson,1,NaN,61,13,2009,17,0,0,45,33,0,[]
21831,http://www.billboard.com/charts/hot-100/1992-0...,1992-02-08,81,Tears In Heaven,Eric Clapton,Tears In HeavenEric Clapton,1,NaN,2,26,1992,6,0,0,73,56,0,[]


In [39]:
print(feature_name[9])
columns_to_drop=feature_name[9:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8]]
print(columns_to_drop)
df_mody=df_array[9].loc[df_array[9]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

graduations
['graduations', 'grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical', 'commercials', 'concerts', 'easter', 'fourthofjuly', 'funerals']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split
384,http://www.billboard.com/charts/hot-100/2016-0...,2016-02-06,96,7 Years,Lukas Graham,7 YearsLukas Graham,1,NaN,2,36,2016,5,0,0,80,0,0,[]
2536,http://www.billboard.com/charts/hot-100/1998-0...,1998-03-28,13,Bitter Sweet Symphony,The Verve,Bitter Sweet SymphonyThe Verve,1,NaN,12,20,1998,13,0,0,79,62,0,[]
2913,http://www.billboard.com/charts/hot-100/2013-0...,2013-05-11,61,Brave,Sara Bareilles,BraveSara Bareilles,1,NaN,23,42,2013,19,1,56,68,0,0,[]
2951,http://www.billboard.com/charts/hot-100/2004-0...,2004-08-28,60,Breakaway,Kelly Clarkson,BreakawayKelly Clarkson,1,NaN,6,46,2004,35,0,0,66,37,1,[]
6644,http://www.billboard.com/charts/hot-100/2010-1...,2010-11-06,57,Firework,Katy Perry,FireworkKaty Perry,1,NaN,1,39,2010,44,0,0,67,72,1,[]
7832,http://www.billboard.com/charts/hot-100/2009-1...,2009-11-28,69,Good Life,OneRepublic,Good LifeOneRepublic,1,NaN,8,35,2009,48,1,511,69,0,0,[]
8029,http://www.billboard.com/charts/hot-100/2000-0...,2000-04-15,81,Graduation (Friends Forever),Vitamin C,Graduation (Friends Forever)Vitamin C,1,NaN,38,12,2000,15,0,0,54,7,0,[]
8302,http://www.billboard.com/charts/hot-100/2014-0...,2014-01-18,98,Happy,Pharrell Williams,HappyPharrell Williams,1,NaN,1,47,2014,3,0,0,73,0,0,[]
8768,http://www.billboard.com/charts/hot-100/2001-0...,2001-06-09,74,Here's To The Night,Eve 6,Here's To The NightEve 6,1,NaN,30,20,2001,23,0,0,57,0,0,[]
8782,http://www.billboard.com/charts/hot-100/1993-1...,1993-10-23,71,Hero,Mariah Carey,HeroMariah Carey,1,NaN,1,30,1993,42,0,0,70,0,1,[]


In [40]:
allSong.loc[allSong['songid']=="Shiny Happy PeopleR.E.M."]

,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,...,karaoke,lullabies,mtvawards,phonecall,politicians,sportingevents,tvshow,weddings,wonoscars,wordcup
19667,http://www.billboard.com/charts/hot-100/1991-0...,1991-07-27,71,Shiny Happy People,R.E.M.,Shiny Happy PeopleR.E.M.,1,NaN,10,15,...,0,0,0,0,0,0,0,0,0,0


In [42]:
allSong.loc[allSong['performer']=="R.E.M."]

2067     0
2538     0
4716     0
5778     0
5903     0
5958     0
6147     0
6363     0
11490    0
12076    0
14026    0
14809    0
17679    0
18005    0
19667    0
20615    0
20769    0
21068    0
21372    0
22531    0
22827    0
25506    0
Name: all_time_greatest_artist, dtype: int64

In [45]:
print(feature_name[10])
columns_to_drop=feature_name[10:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9]]
print(columns_to_drop)
df_mody=df_array[10].loc[df_array[10]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

grammys
['grammys', 'halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical', 'commercials', 'concerts', 'easter', 'fourthofjuly', 'funerals', 'graduations']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split
43,http://www.billboard.com/charts/hot-100/1991-0...,1991-06-29,53,(Everything I Do) I Do It For You,Bryan Adams,(Everything I Do) I Do It For YouBryan Adams,1,NaN,1,22,1991,26,0,0,65,50,1,[]
163,http://www.billboard.com/charts/hot-100/1968-0...,1968-01-27,67,(Sittin' On) The Dock Of The Bay,Otis Redding,(Sittin' On) The Dock Of The BayOtis Redding,1,NaN,1,16,1968,4,0,0,69,62,0,[]
486,http://www.billboard.com/charts/hot-100/1964-0...,1964-07-18,21,A Hard Day's Night,The Beatles,A Hard Day's NightThe Beatles,1,NaN,1,13,1964,29,0,0,67,57,1,[]
798,http://www.billboard.com/charts/hot-100/1979-0...,1979-07-07,77,After The Love Has Gone,"Earth, Wind & Fire","After The Love Has GoneEarth, Wind & Fire",1,NaN,2,17,1979,27,0,0,61,50,0,[]
810,http://www.billboard.com/charts/hot-100/2000-1...,2000-11-18,72,Again,Lenny Kravitz,AgainLenny Kravitz,1,NaN,4,32,2000,46,0,0,55,23,0,[]
812,http://www.billboard.com/charts/hot-100/1980-0...,1980-05-03,54,Against The Wind,Bob Seger,Against The WindBob Seger,1,NaN,5,17,1980,18,0,0,60,36,1,[]
841,http://www.billboard.com/charts/hot-100/2014-0...,2014-03-15,96,Ain't It Fun,Paramore,Ain't It FunParamore,1,NaN,10,24,2014,11,0,0,71,0,0,[]
854,http://www.billboard.com/charts/hot-100/2006-0...,2006-06-24,19,Ain't No Other Man,Christina Aguilera,Ain't No Other ManChristina Aguilera,1,NaN,6,20,2006,25,0,0,60,48,0,[]
859,http://www.billboard.com/charts/hot-100/1971-0...,1971-07-17,88,Ain't No Sunshine,Bill Withers,Ain't No SunshineBill Withers,1,NaN,3,16,1971,28,0,0,75,54,0,[]
879,http://www.billboard.com/charts/hot-100/1968-0...,1968-04-13,75,Ain't Nothing Like The Real Thing,Marvin Gaye & Tammi Terrell,Ain't Nothing Like The Real ThingMarvin Gaye &...,1,NaN,8,13,1968,15,0,0,48,34,0,[]


In [46]:
print(feature_name[11])
columns_to_drop=feature_name[11:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9],feature_name[10]]
print(columns_to_drop)
df_mody=df_array[11].loc[df_array[11]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

halloween
['halloween', 'inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical', 'commercials', 'concerts', 'easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split
4858,http://www.billboard.com/charts/hot-100/1976-0...,1976-07-03,87,Devil Woman,Cliff Richard,Devil WomanCliff Richard,1,NaN,6,22,1976,27,0,0,48,57,0,[]
18851,http://www.billboard.com/charts/hot-100/1978-0...,1978-05-06,93,Runnin' With The Devil,Van Halen,Runnin' With The DevilVan Halen,1,NaN,84,4,1978,18,0,0,68,43,0,[]
23644,http://www.billboard.com/charts/hot-100/1984-0...,1984-02-11,20,Thriller,Michael Jackson,ThrillerMichael Jackson,1,NaN,4,17,1984,6,3,10780,67,73,1,[]
25193,http://www.billboard.com/charts/hot-100/1978-0...,1978-03-25,76,Werewolves Of London,Warren Zevon,Werewolves Of LondonWarren Zevon,1,NaN,21,12,1978,12,0,0,58,62,0,[]


In [47]:
print(feature_name[12])
columns_to_drop=feature_name[12:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9],feature_name[10],feature_name[11]]
print(columns_to_drop)
df_mody=df_array[12].loc[df_array[12]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

inmovieclueless
['inmovieclueless', 'inmovies', 'inplays', 'intvshows', 'karaoke', 'lullabies', 'mtvawards', 'phonecall', 'politicians', 'sportingevents', 'tvshow', 'weddings', 'wonoscars', 'wordcup', 'artistdied', 'christmas', 'church', 'classical', 'commercials', 'concerts', 'easter', 'fourthofjuly', 'funerals', 'graduations', 'grammys', 'halloween']


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split
963,http://www.billboard.com/charts/hot-100/1975-1...,1975-12-20,85,All By Myself,Eric Carmen,All By MyselfEric Carmen,1,NaN,2,19,1975,51,0,0,57,50,0,[]
6444,http://www.billboard.com/charts/hot-100/1980-1...,1980-12-06,87,Fashion,David Bowie,FashionDavid Bowie,1,NaN,70,9,1980,49,0,0,50,31,0,[]
7517,http://www.billboard.com/charts/hot-100/1983-1...,1983-12-17,80,Girls Just Want To Have Fun,Cyndi Lauper,Girls Just Want To Have FunCyndi Lauper,1,NaN,2,25,1983,50,0,0,73,68,0,[]
12443,http://www.billboard.com/charts/hot-100/1995-1...,1995-12-16,84,Just A Girl,No Doubt,Just A GirlNo Doubt,1,NaN,23,29,1995,50,0,0,61,52,0,[]
19689,http://www.billboard.com/charts/hot-100/1993-1...,1993-10-09,68,Shoop,Salt-N-Pepa,ShoopSalt-N-Pepa,1,NaN,4,25,1993,40,0,0,65,59,0,[]


In [69]:
print(feature_name[13])
columns_to_drop=feature_name[14:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9],feature_name[10],feature_name[11],feature_name[12]]
# print(columns_to_drop)
df_mody=df_array[13].loc[df_array[13]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

inmovies


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,inmovies
43,http://www.billboard.com/charts/hot-100/1991-0...,1991-06-29,53,(Everything I Do) I Do It For You,Bryan Adams,(Everything I Do) I Do It For YouBryan Adams,1,NaN,1,22,1991,26,0,0,65,50,1,[],1
202,http://www.billboard.com/charts/hot-100/1966-1...,1966-12-10,89,(We Ain't Got) Nothin' Yet,Blues Magoos,(We Ain't Got) Nothin' YetBlues Magoos,1,NaN,5,14,1966,49,0,0,33,35,0,[],1
219,http://www.billboard.com/charts/hot-100/1999-0...,1999-09-18,62,(You Drive Me) Crazy,Britney Spears,(You Drive Me) CrazyBritney Spears,1,NaN,10,20,1999,37,0,0,60,45,0,[],1
250,http://www.billboard.com/charts/hot-100/2005-0...,2005-02-12,100,1 Thing,Amerie,1 ThingAmerie,1,NaN,8,20,2005,6,0,0,58,59,0,[],1
266,http://www.billboard.com/charts/hot-100/1988-0...,1988-06-04,76,1-2-3,Gloria Estefan & Miami Sound Machine,1-2-3Gloria Estefan & Miami Sound Machine,1,NaN,3,19,1988,22,0,0,36,12,0,[],1
412,http://www.billboard.com/charts/hot-100/1983-1...,1983-12-10,74,99 Luftballons,Nena,99 LuftballonsNena,1,NaN,2,23,1983,49,0,0,56,70,0,[],1
486,http://www.billboard.com/charts/hot-100/1964-0...,1964-07-18,21,A Hard Day's Night,The Beatles,A Hard Day's NightThe Beatles,1,NaN,1,13,1964,29,0,0,67,57,1,[],1
490,http://www.billboard.com/charts/hot-100/1972-0...,1972-02-19,84,A Horse With No Name,America,A Horse With No NameAmerica,1,NaN,1,14,1972,7,0,0,75,33,0,[],1
499,http://www.billboard.com/charts/hot-100/1986-0...,1986-06-21,85,A Kind Of Magic,Queen,A Kind Of MagicQueen,1,NaN,42,11,1986,25,0,0,60,35,0,[],1
583,http://www.billboard.com/charts/hot-100/1986-0...,1986-08-09,71,A Matter Of Trust,Billy Joel,A Matter Of TrustBilly Joel,1,NaN,10,18,1986,32,0,0,49,40,1,[],1


In [70]:
print(feature_name[14])
columns_to_drop=feature_name[15:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9],feature_name[10],feature_name[11],feature_name[12],feature_name[13]]
# print(columns_to_drop)
df_mody=df_array[14].loc[df_array[14]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody

inplays


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,inplays
1351,http://www.billboard.com/charts/hot-100/2007-0...,2007-01-13,98,And I Am Telling You I'm Not Going,Jennifer Hudson,And I Am Telling You I'm Not GoingJennifer Hudson,1,NaN,60,13,2007,2,0,0,57,34,0,[],1
5966,http://www.billboard.com/charts/hot-100/1969-1...,1969-10-25,61,Eli's Coming,Three Dog Night,Eli's ComingThree Dog Night,1,NaN,10,14,1969,43,0,0,28,33,1,[],1
9179,http://www.billboard.com/charts/hot-100/1974-0...,1974-09-14,89,"Honey, Honey",ABBA,"Honey, HoneyABBA",1,NaN,27,10,1974,37,0,0,48,34,0,[],1
9908,http://www.billboard.com/charts/hot-100/2013-0...,2013-01-12,69,I Dreamed A Dream,Anne Hathaway,I Dreamed A DreamAnne Hathaway,1,NaN,69,3,2013,2,0,0,53,0,0,[],1
10348,http://www.billboard.com/charts/hot-100/1959-0...,1959-05-30,60,I Only Have Eyes For You,The Flamingos,I Only Have Eyes For YouThe Flamingos,1,NaN,11,13,1959,22,0,0,56,37,0,[],1
11255,http://www.billboard.com/charts/hot-100/1969-0...,1969-07-19,90,I'm Free,The Who,I'm FreeThe Who,1,NaN,37,8,1969,29,0,0,40,0,0,[],1
13063,http://www.billboard.com/charts/hot-100/2016-0...,2016-01-30,40,Lazarus,David Bowie,LazarusDavid Bowie,1,NaN,40,1,2016,4,0,0,58,0,0,[],1
14612,http://www.billboard.com/charts/hot-100/1959-0...,1959-08-22,59,Mack The Knife,Bobby Darin,Mack The KnifeBobby Darin,1,NaN,1,26,1959,34,0,0,52,56,1,[],1
16692,http://www.billboard.com/charts/hot-100/2013-0...,2013-01-19,97,On My Own,Samantha Barks,On My OwnSamantha Barks,1,NaN,97,1,2013,3,0,0,53,0,0,[],1
17489,http://www.billboard.com/charts/hot-100/1969-0...,1969-04-05,73,Pinball Wizard,The Who,Pinball WizardThe Who,1,NaN,19,11,1969,14,0,0,67,24,0,[],1


In [77]:
print(feature_name[15])
columns_to_drop=feature_name[16:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9],feature_name[10],feature_name[11],feature_name[12],feature_name[13],feature_name[14]]
# print(columns_to_drop)
df_mody=df_array[15].loc[df_array[15]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody.sort_values(by='year',ascending =True)

intvshows


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,intvshows
2427,http://www.billboard.com/charts/hot-100/1960-0...,1960-01-16,74,Beyond The Sea,Bobby Darin,Beyond The SeaBobby Darin,1,NaN,6,14,1960,2,0,0,58,63,1,[],1
12331,http://www.billboard.com/charts/hot-100/1962-0...,1962-03-03,81,Johnny Angel,Shelley Fabares,Johnny AngelShelley Fabares,1,NaN,1,15,1962,9,0,0,28,34,0,[],1
19495,http://www.billboard.com/charts/hot-100/1962-0...,1962-03-03,90,She Can't Find Her Keys,Paul Petersen,She Can't Find Her KeysPaul Petersen,1,NaN,19,12,1962,9,0,0,9,3,0,[],1
3032,http://www.billboard.com/charts/hot-100/1962-0...,1962-06-23,94,Bring It On Home To Me,Sam Cooke,Bring It On Home To MeSam Cooke,1,NaN,13,11,1962,25,1,21,59,69,0,[],1
25058,http://www.billboard.com/charts/hot-100/1965-0...,1965-08-14,80,We Gotta Get Out Of This Place,The Animals,We Gotta Get Out Of This PlaceThe Animals,1,NaN,13,11,1965,32,0,0,49,37,0,[],1
11846,http://www.billboard.com/charts/hot-100/1965-1...,1965-12-25,77,It Was A Very Good Year,Frank Sinatra,It Was A Very Good YearFrank Sinatra,1,NaN,28,8,1965,51,0,0,59,44,0,[],1
3715,http://www.billboard.com/charts/hot-100/1966-0...,1966-08-27,66,Cherish,The Association,CherishThe Association,1,NaN,1,14,1966,34,0,0,35,41,0,[],1
7372,http://www.billboard.com/charts/hot-100/1967-0...,1967-09-02,88,Get Together,The Youngbloods,Get TogetherThe Youngbloods,1,NaN,5,25,1967,35,1,616,51,41,0,[],1
8286,http://www.billboard.com/charts/hot-100/1967-0...,1967-02-11,79,Happy Together,The Turtles,Happy TogetherThe Turtles,1,NaN,1,15,1967,6,0,0,72,44,0,[],1
3239,http://www.billboard.com/charts/hot-100/1967-0...,1967-02-04,94,California Nights,Lesley Gore,California NightsLesley Gore,1,NaN,16,14,1967,5,0,0,25,20,0,[],1


In [78]:
print(feature_name[16])
columns_to_drop=feature_name[17:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9],feature_name[10],feature_name[11],feature_name[12],feature_name[13],feature_name[14],feature_name[15]]
# print(columns_to_drop)
df_mody=df_array[16].loc[df_array[16]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody.sort_values(by='year',ascending =True)

karaoke


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,karaoke
26494,http://www.billboard.com/charts/hot-100/1966-0...,1966-08-20,52,Yellow Submarine,The Beatles,Yellow SubmarineThe Beatles,1,NaN,2,9,1966,33,0,0,64,54,1,[],1
12734,http://www.billboard.com/charts/hot-100/1973-0...,1973-01-27,54,Killing Me Softly With His Song,Roberta Flack,Killing Me Softly With His SongRoberta Flack,1,NaN,1,16,1973,4,0,0,71,45,0,[],1
4598,http://www.billboard.com/charts/hot-100/1976-1...,1976-12-11,86,Dancing Queen,ABBA,Dancing QueenABBA,1,NaN,1,22,1976,50,0,0,71,59,0,[],1
24994,http://www.billboard.com/charts/hot-100/1979-0...,1979-04-28,63,We Are Family,Sister Sledge,We Are FamilySister Sledge,1,NaN,2,19,1979,17,0,0,67,35,0,[],1
5470,http://www.billboard.com/charts/hot-100/1981-1...,1981-10-31,56,Don't Stop Believin',Journey,Don't Stop Believin'Journey,1,NaN,9,16,1981,44,0,0,78,74,0,[],1
2188,http://www.billboard.com/charts/hot-100/1983-0...,1983-02-26,78,Beat It,Michael Jackson,Beat ItMichael Jackson,1,NaN,1,25,1983,8,0,0,77,66,1,[],1
7517,http://www.billboard.com/charts/hot-100/1983-1...,1983-12-17,80,Girls Just Want To Have Fun,Cyndi Lauper,Girls Just Want To Have FunCyndi Lauper,1,NaN,2,25,1983,50,0,0,73,68,0,[],1
23644,http://www.billboard.com/charts/hot-100/1984-0...,1984-02-11,20,Thriller,Michael Jackson,ThrillerMichael Jackson,1,NaN,4,17,1984,6,3,10780,67,73,1,[],1
25245,http://www.billboard.com/charts/hot-100/1988-0...,1988-02-20,67,What A Wonderful World,Louis Armstrong,What A Wonderful WorldLouis Armstrong,1,NaN,32,11,1988,7,0,0,64,52,0,[],1
14356,http://www.billboard.com/charts/hot-100/1989-0...,1989-09-02,84,Love Shack,The B-52s,Love ShackThe B-52s,1,NaN,3,27,1989,35,0,0,66,23,0,[],1


In [81]:
print(feature_name[17])
columns_to_drop=feature_name[18:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9],feature_name[10],feature_name[11],feature_name[12],feature_name[13],feature_name[14],feature_name[15],feature_name[16]]
df_mody=df_array[17].loc[df_array[17]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody.sort_values(by='year',ascending =True)

lullabies


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,lullabies
14573,http://www.billboard.com/charts/hot-100/1994-0...,1994-03-26,90,"Lullabye (Goodnight, My Angel)",Billy Joel,"Lullabye (Goodnight, My Angel)Billy Joel",1,NaN,77,8,1994,12,0,0,52,41,1,[],1


In [82]:
print(feature_name[18])
columns_to_drop=feature_name[19:26]+[feature_name[0], feature_name[1], feature_name[2], feature_name[3],feature_name[4],feature_name[5],feature_name[6],feature_name[7],feature_name[8],feature_name[9],feature_name[10],feature_name[11],feature_name[12],feature_name[13],feature_name[14],feature_name[15],feature_name[16]]
# print(columns_to_drop)
df_mody=df_array[18].loc[df_array[18]['strict_rule']>0].drop(columns_to_drop, axis=1)
df_mody.sort_values(by='year',ascending =True)

mtvawards


,url,weekid,week_position,song,performer,songid,instance,previous_week_position,peak_position,weeks_on_chart,year,week,jump_time,max_jump_duration,strict_rule,lenient_rule,all_time_greatest_artist,split,lullabies,mtvawards
12934,http://www.billboard.com/charts/hot-100/1975-0...,1975-01-04,98,Lady Marmalade,Labelle,Lady MarmaladeLabelle,1,NaN,1,18,1975,1,0,0,59,39,0,[],0,1
3756,http://www.billboard.com/charts/hot-100/1983-0...,1983-06-04,74,China Girl,David Bowie,China GirlDavid Bowie,1,NaN,10,18,1983,22,0,0,57,38,0,[],0,1
18614,http://www.billboard.com/charts/hot-100/1983-0...,1983-09-10,86,Rockit,Herbie Hancock,RockitHerbie Hancock,1,NaN,71,9,1983,36,0,0,50,0,0,[],0,1
7517,http://www.billboard.com/charts/hot-100/1983-1...,1983-12-17,80,Girls Just Want To Have Fun,Cyndi Lauper,Girls Just Want To Have FunCyndi Lauper,1,NaN,2,25,1983,50,0,0,73,68,0,[],0,1
26840,http://www.billboard.com/charts/hot-100/1984-0...,1984-03-10,57,You Might Think,The Cars,You Might ThinkThe Cars,1,NaN,7,17,1984,10,0,0,55,42,0,[],0,1
23644,http://www.billboard.com/charts/hot-100/1984-0...,1984-02-11,20,Thriller,Michael Jackson,ThrillerMichael Jackson,1,NaN,4,17,1984,6,3,10780,67,73,1,[],0,1
12422,http://www.billboard.com/charts/hot-100/1984-0...,1984-01-14,47,Jump,Van Halen,JumpVan Halen,1,NaN,1,21,1984,2,0,0,76,0,0,[],0,1
23051,http://www.billboard.com/charts/hot-100/1985-1...,1985-11-30,79,The Sun Always Shines on T.V.,a-ha,The Sun Always Shines on T.V.a-ha,1,NaN,20,17,1985,48,0,0,58,31,0,[],0,1
21698,http://www.billboard.com/charts/hot-100/1985-0...,1985-07-13,91,Take On Me,a-ha,Take On Mea-ha,1,NaN,1,27,1985,28,0,0,83,65,0,[],0,1
11382,http://www.billboard.com/charts/hot-100/1985-0...,1985-02-16,54,I'm On Fire,Bruce Springsteen,I'm On FireBruce Springsteen,1,NaN,6,20,1985,7,0,0,71,0,1,[],0,1


In [100]:
for element in df_mody['performer'].tolist():
    print(element)
    print(allSong.loc[allSong['performer']==element]['mtvawards'].tolist())
    print(allSong.loc[allSong['performer']==element]['strict_rule'].tolist())

Nicki Minaj
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[64, 0, 59, 62, 56, 51, 33, 66, 53, 68, 59, 59, 54]
Lady Gaga
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[61, 71, 65, 68, 65, 51, 64, 0, 59, 57, 56, 81, 62, 69, 68, 52, 75, 61, 60, 61, 62]
David Bowie
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[44, 52, 47, 0, 63, 57, 0, 57, 50, 0, 46, 58, 69, 65, 0, 72, 58, 71, 72, 35, 0, 39, 40, 40, 60]
Linkin Park
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]
[72, 63, 70, 73, 67, 66, 83, 53, 67, 64, 58, 67, 81, 73, 65, 64, 53, 56, 76]
David Bowie
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[44, 52, 47, 0, 63, 57, 0, 57, 50, 0, 46, 58, 69, 65, 0, 72, 58, 71, 72, 35, 0, 39, 40, 40, 60]
Avril Lavigne
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[32, 75, 58, 67, 55, 67, 47, 66, 0, 55, 68, 62, 60, 73, 57, 66, 63, 56]
U2
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[41, 83, 58]

In [60]:
early_song=allSong.loc[allSong['year']<=1970]
early_song.shape

(8408, 44)

In [64]:
early_song_valid=early_song.loc[early_song['strict_rule']>0]
early_song_valid.shape

(2277, 44)

In [65]:
early_song_valid.loc[early_song_valid['inmovies']==1].shape

(99, 44)

In [66]:
early_song_notvalid=early_song.loc[early_song['strict_rule']==0]
early_song_notvalid.shape

(6131, 44)

In [67]:
early_song_notvalid.loc[early_song_notvalid['inmovies']==1].shape

(75, 44)

In [ ]:
olde_song 